Install libraries:

In [ ]:
!pip install sentencepiece

In [4]:
!pip install --upgrade transformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.1.1)


In [89]:
!pip install rouge_score

Import libraries:

In [90]:
import torch
import nltk
nltk.download('punkt')

from rouge_score import rouge_scorer

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Custom defined functions:

In [29]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Define parameters:

In [30]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Example:

In [38]:
txt = '''In optics, dispersion is the phenomenon in which the phase velocity of a wave depends on its frequency.[1] Media having this common property may be termed dispersive media. 
Sometimes the term chromatic dispersion is used for specificity. Although the term is used in the field of optics to describe light and other electromagnetic waves, dispersion in the 
same sense can apply to any sort of wave motion such as acoustic dispersion in the case of sound and seismic waves, in gravity waves (ocean waves), and for telecommunication signals 
along transmission lines (such as coaxial cable) or optical fiber. In optics, one important and familiar consequence of dispersion is the change in the angle of refraction of different 
colors of light,[2] as seen in the spectrum produced by a dispersive prism and in chromatic aberration of lenses. Design of compound achromatic lenses, in which chromatic aberration is 
largely cancelled, uses a quantification of a glass's dispersion given by its Abbe number V, where lower Abbe numbers correspond to greater dispersion over the visible spectrum. 
In some applications such as telecommunications, the absolute phase of a wave is often not important but only the propagation of wave packets or "pulses"; in that case one is interested 
only in variations of group velocity with frequency, so-called group-velocity dispersion.
'''

Create abstractive summary:

In [85]:
%%time
summ_list = []

for i in sent_tokenize(txt):
  summ_list.append(get_response(i,1)[0])

summary = ' '.join(summ_list)  

CPU times: user 1min 20s, sys: 613 ms, total: 1min 21s
Wall time: 1min 21s


Abstractive summary:

In [86]:
summary

'The phase velocity of a wave depends on its Frequency. This common property may be called dispersive media. The term chromatic dispersion can be used for specificity. The term dispersion can be used to describe any type of wave motion such as acoustic dispersion in the case of sound and seismic waves, in gravity waves, and for telecommunication signals along transmission lines. The change in the angle of refraction of different colors of light is one of the consequences of dispersion. The design of a compound achromatic lens uses a quantification of a glass\'s dispersion given by its Abbe number V, where lower Abbe numbers correspond to greater dispersion over the visible spectrum. The propagation of wave packets or "pulses" is more important than the absolute phase of a wave in some applications.'

ROUGE:

In [93]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
scores = scorer.score(summary, txt)
scores = round(100*list(list(scores.values())[0])[2])

print('ROUGE:', (str(scores) + ' %'))

ROUGE: 61 %


Define compression rate:

In [87]:
print('Compression rate:', round(len(txt)/len(nltk.word_tokenize(' '.join(summary))),1)) 

Compression rate: 2.0
